In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, BertPreTrainedModel, DistilBertTokenizer, DistilBertModel, DistilBertPreTrainedModel
from sklearn.metrics import mean_squared_error, cohen_kappa_score

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [ ]:
df = pd.read_csv("essay_set_8.csv")
df

,Unnamed: 0,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6
0,12255,20716,8,A long time ago when I was in third grade I h...,18.0,16.0,34.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0
1,12256,20717,8,Softball has to be one of the single most gre...,21.0,26.0,46.0,5.0,4.0,5.0,4.0,4.0,4.0,6.0,5.0,6.0,6.0,5.0,5.0
2,12257,20718,8,"Some people like making people laugh, I love ...",15.0,20.0,40.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,5.0,4.0,4.0,4.0
3,12258,20719,8,"""LAUGHTER"" @CAPS1 I hang out with my friends...",12.0,20.0,30.0,3.0,3.0,3.0,3.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0
4,12259,20721,8,Well ima tell a story about the time i got @CA...,11.0,15.0,26.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,12973,21626,8,In most stories mothers and daughters are eit...,17.0,18.0,35.0,4.0,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,3.0
719,12974,21628,8,I never understood the meaning laughter is th...,15.0,17.0,32.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,3.0,4.0,4.0,4.0,3.0
720,12975,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20.0,26.0,40.0,4.0,4.0,4.0,4.0,4.0,4.0,6.0,5.0,5.0,5.0,5.0,5.0
721,12976,21630,8,Trippin' on fen...,20.0,20.0,40.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [ ]:
pip install language_tool_python

In [ ]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

In [ ]:
def get_grammar_spelling_errors(text):
    matches = tool.check(text)
    return matches

# text = "I am not a patience person, like I can’t sit in a sit for more than five minutes, but there was one time I was patience and i bet very alse was to that right riding a bike where out training wheels. When I was five learned how to ride a bike I just keeped falling of at one piont my bike  flip and I allmost broke my jaw but i didn’t. Finally after @NUM1 weeks of onstint falling I rode for @NUM2 minutes without falling and finally learn how to riken a bike."
# matches = get_grammar_spelling_errors(text)
# print(f"Grammar errors: {len(matches)}")

# for match in matches:
#     print(f"Error: {match.message} at position {match.offset}")


In [ ]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load Spacy model and a sentence embedding model
nlp = spacy.load("en_core_web_sm")
embedder = pipeline("feature-extraction", model="bert-base-uncased", tokenizer="bert-base-uncased", truncation= True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")



def get_sentence_embeddings(text, max_length=512):
    # Tokenize the text into sentences
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    embeddings = []
    for sentence in sentences:
        tokens = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**tokens)
            sentence_embedding = outputs.last_hidden_state.mean(dim=1).numpy().flatten()
        embeddings.append(sentence_embedding)

    return embeddings

def sentence_similarity(text):
    # Get embeddings for the text
    embeddings = get_sentence_embeddings(text)

    # Calculate cosine similarity between consecutive sentence embeddings
    similarities = [cosine_similarity([embeddings[i]], [embeddings[i+1]])[0, 0] for i in range(len(embeddings) - 1)]
    avg_similarity = np.mean(similarities) if similarities else 0

    return avg_similarity



# def sentence_similarity(text):
#     # Tokenize the text into sentences
#     doc = nlp(text)
#     sentences = [sent.text for sent in doc.sents]

#     # Generate embeddings for each sentence
#     embeddings = [np.mean(embedder(sentence)[0], axis=0) for sentence in sentences]

#     # Calculate cosine similarity between consecutive sentences
#     similarities = [cosine_similarity([embeddings[i]], [embeddings[i+1]])[0, 0] for i in range(len(embeddings) - 1)]
#     avg_similarity = np.mean(similarities) if similarities else 0


#     return avg_similarity

# Example essay
# essay = "The climate is changing rapidly. This change is mainly due to human activities. It affects weather patterns and ecosystems worldwide."

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
essay = df["essay"][2]
# Extract coherence features
features = sentence_similarity(essay)
print("Adjencent sentence similarity:", features)

Adjencent sentence similarity: 0.71224356


In [ ]:
import spacy
from collections import Counter
import numpy as np

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

def extract_features(text):
    doc = nlp(text)

    # Lexical Cohesion Features
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    lemma_counts = Counter(lemmas)

    lexical_repetition_ratio = sum(count for count in lemma_counts.values() if count > 1) / len(tokens) if tokens else 0
    unique_lemmas_count = len(lemma_counts)

    # Pronoun Usage
    pronouns = [token.text.lower() for token in doc if token.pos_ == "PRON"]
    pronoun_count = len(pronouns)

    # Conjunctions and Connectives
    conjunctions = {"and", "or", "but", "because", "although", "however", "therefore", "moreover", "thus", "furthermore", "then"}
    conjunction_count = sum(1 for token in doc if token.text.lower() in conjunctions)

    # Entity Coherence
    entities = [ent.text.lower() for ent in doc.ents]
    entity_counts = Counter(entities)
    entity_repetition_count = sum(count for count in entity_counts.values() if count > 1)

    # Sentence-Level Features
    sentence_lengths = [len(sent) for sent in doc.sents]
    avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
    sentence_length_std = np.std(sentence_lengths) if sentence_lengths else 0

    # Paragraph-Level Features
    paragraphs = text.split('\n\n')
    paragraph_lengths = [len(p.split()) for p in paragraphs]
    avg_paragraph_length = np.mean(paragraph_lengths) if paragraph_lengths else 0

    features = {
        "lexical_repetition_ratio": lexical_repetition_ratio,
        "unique_lemmas_count": unique_lemmas_count,
        "pronoun_count": pronoun_count,
        "conjunction_count": conjunction_count,
        "entity_repetition_count": entity_repetition_count,
        "avg_sentence_length": avg_sentence_length,
        "sentence_length_std": sentence_length_std,
        "avg_paragraph_length": avg_paragraph_length
    }

    return features



In [ ]:
# Example essay
essay = """
The rapid changes in climate are primarily caused by human activities.
This is evident in the increased carbon emissions from vehicles and industries.
Furthermore, deforestation plays a significant role in disrupting ecological balance.
Therefore, it is crucial to adopt sustainable practices to mitigate these impacts.
"""

# Extract features
features = extract_features(essay)
print("Handcrafted Features:", features)

Handcrafted Features: {'lexical_repetition_ratio': 0.15625, 'unique_lemmas_count': 28, 'pronoun_count': 2, 'conjunction_count': 3, 'entity_repetition_count': 0, 'avg_sentence_length': 14.0, 'sentence_length_std': 0.7071067811865476, 'avg_paragraph_length': 45.0}


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')

def count_discourse_connectives(essay):
    """
    Count the occurrences of discourse connectives in a given essay.

    Parameters:
    - essay (str): The text of the essay.
    - discourse_connectives (list): List of discourse connectives to look for.

    Returns:
    - connective_counts (Counter): A Counter object with the count of each connective.
    - total_count (int): Total count of discourse connectives found.
    """

    # Define discourse connectives
    discourse_connectives = [
        "furthermore", "moreover", "in addition", "besides", "also", "similarly",
        "as well as", "equally important", "not only", "but also", "however",
        "although", "though", "on the other hand", "nevertheless", "nonetheless",
        "whereas", "while", "despite", "in contrast", "alternatively", "therefore",
        "consequently", "thus", "as a result", "because", "since", "due to",
        "hence", "if", "unless", "provided that", "in case", "as long as",
        "on the condition that", "then", "subsequently", "earlier", "later",
        "meanwhile", "before", "after", "during", "once", "until", "when",
        "indeed", "in fact", "certainly", "especially", "particularly",
        "importantly", "in conclusion", "to sum up", "overall", "in summary",
        "thus", "consequently", "ultimately", "for example", "for instance",
        "such as", "to illustrate", "namely", "in other words", "that is to say",
        "to put it another way"
    ]

    # Tokenize the essay
    tokens = word_tokenize(essay.lower())

    # Count occurrences of discourse connectives
    connective_counts = Counter(token for token in tokens if token in discourse_connectives)
    total_count = sum(connective_counts.values())

    return connective_counts, total_count





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Example usage
essay = df["essay"][2]
connective_counts, total_count = count_discourse_connectives(essay)

print("Discourse Connective Counts:", connective_counts)
print("Total Count:", total_count)

Discourse Connective Counts: Counter({'while': 1, 'since': 1, 'also': 1})
Total Count: 3


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

# Ensure required NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

def calculate_vocabulary_richness(text):
    """
    Calculate the Type-Token Ratio (TTR) to measure vocabulary richness in a given text.

    Parameters:
    text (str): The input text for which to calculate vocabulary richness.

    Returns:
    float: The Type-Token Ratio (TTR) representing the vocabulary richness.
    """
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove punctuation and stopwords
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Calculate the number of unique words and total words
    unique_words = set(tokens)
    total_words = len(tokens)
    unique_word_count = len(unique_words)

    # Calculate Type-Token Ratio (TTR)
    if total_words > 0:
        ttr = unique_word_count / total_words
    else:
        ttr = 0.0

    return ttr




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Example usage
text = df["essay"][4]
vocabulary_richness = calculate_vocabulary_richness(text)
print(f"Type-Token Ratio (TTR): {vocabulary_richness:.2f}")

Type-Token Ratio (TTR): 0.69


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# def get_embedding(text, model, tokenizer):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
#     outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# def calculate_semantic_relevance(prompt, essay):
#     prompt_embedding = get_embedding(prompt, model, tokenizer)
#     essay_embedding = get_embedding(essay, model, tokenizer)

#     similarity = cosine_similarity(prompt_embedding, essay_embedding)
#     return similarity[0, 0]


def get_embedding(text, model, tokenizer):
    # Tokenize text
    inputs = tokenizer(text, return_tensors='pt', truncation=False, padding=False)
    input_ids = inputs['input_ids'].squeeze()
    attention_mask = inputs['attention_mask'].squeeze()

    # Process text in chunks
    chunk_size = 512
    embeddings = []
    for i in range(0, len(input_ids), chunk_size):
        chunk_ids = input_ids[i:i + chunk_size].unsqueeze(0)
        chunk_attention_mask = attention_mask[i:i + chunk_size].unsqueeze(0)
        with torch.no_grad():
            outputs = model(input_ids=chunk_ids, attention_mask=chunk_attention_mask)
        chunk_embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        embeddings.append(chunk_embedding)

    # Aggregate chunk embeddings
    aggregated_embedding = np.mean(np.vstack(embeddings), axis=0)
    return aggregated_embedding

def calculate_semantic_relevance(prompt, essay):
    prompt_embedding = get_embedding(prompt, model, tokenizer)
    essay_embedding = get_embedding(essay, model, tokenizer)

    prompt_embedding = prompt_embedding.reshape(1, -1)
    essay_embedding = essay_embedding.reshape(1, -1)

    similarity = cosine_similarity(prompt_embedding, essay_embedding)
    return similarity[0, 0]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Sample prompt and essay
prompt = '''Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.
Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.
'''
essay = df["essay"][1]

# Calculate semantic relevance
score = calculate_semantic_relevance(prompt_8, essay)
print(f"Semantic Relevance Score: {score:.2f}")


Semantic Relevance Score: 0.78


In [ ]:
pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00


In [ ]:
import textstat
text = df["essay"][2]

# Calculate readability scores
print("Flesch Reading Ease:", textstat.flesch_reading_ease(text))
print("Flesch-Kincaid Grade Level:", textstat.flesch_kincaid_grade(text))
print("Gunning Fog Index:", textstat.gunning_fog(text))
print("SMOG Index:", textstat.smog_index(text))
print("Automated Readability Index:", textstat.automated_readability_index(text))
print("Coleman-Liau Index:", textstat.coleman_liau_index(text))


Flesch Reading Ease: 67.28
Flesch-Kincaid Grade Level: 9.0
Gunning Fog Index: 10.97
SMOG Index: 11.0
Automated Readability Index: 10.9
Coleman-Liau Index: 9.23


In [ ]:
def calculate_all_features(prompt,essay):
  num_grammatical_errors = len(get_grammar_spelling_errors(essay))
  adjacent_sentence_similarity = sentence_similarity(essay)
  features = extract_features(essay)
  connective_counts, total_count = count_discourse_connectives(essay)
  vocabulary_richness = calculate_vocabulary_richness(essay)
  semantic_relevance = calculate_semantic_relevance(prompt, essay)
  #readability scores
  flesch_reading_ease = textstat.flesch_reading_ease(essay)
  flesch_kincaid_grade = textstat.flesch_kincaid_grade(essay)
  gunning_fog_index = textstat.gunning_fog(essay)
  smog_index = textstat.smog_index(essay)
  automated_readability_index = textstat.automated_readability_index(essay)
  coleman_liau_index = textstat.coleman_liau_index(essay)

  return {
      "num_grammatical_errors": num_grammatical_errors,
      "adjacent_sentence_similarity": adjacent_sentence_similarity,
      "lexical_repetition_ratio": features["lexical_repetition_ratio"],
      "unique_lemmas_count": features["unique_lemmas_count"],
      "pronoun_count": features["pronoun_count"],
      "avg_sentence_length": features["avg_sentence_length"],
      "sentence_length_std": features["sentence_length_std"],
      "discourse_connective_count": total_count,
      "vocabulary_richness": vocabulary_richness,
      "semantic_relevance": semantic_relevance,
      "flesch_reading_ease": flesch_reading_ease,
      "flesch_kincaid_grade": flesch_kincaid_grade,
      "gunning_fog_index": gunning_fog_index,
      "smog_index": smog_index,
      "automated_readability_index": automated_readability_index,
      "coleman_liau_index": coleman_liau_index
  }


In [ ]:
calculate_all_features(prompt_6, df["essay"][0])

{'num_grammatical_errors': 3,
 'adjacent_sentence_similarity': 0.6585304,
 'lexical_repetition_ratio': 0.21153846153846154,
 'unique_lemmas_count': 45,
 'pronoun_count': 6,
 'avg_sentence_length': 19.0,
 'sentence_length_std': 4.47213595499958,
 'discourse_connective_count': 3,
 'vocabulary_richness': 0.8688524590163934,
 'semantic_relevance': 0.7863396,
 'flesch_reading_ease': 59.33,
 'flesch_kincaid_grade': 10.0,
 'gunning_fog_index': 10.74,
 'smog_index': 11.5,
 'automated_readability_index': 10.9,
 'coleman_liau_index': 9.4}

In [ ]:
prompt_1 = '''More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends.
Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.'''

prompt_2 = ''' "All of us can think of a book that we hope none of our children or any other children have taken off the shelf. But if I have the right to remove that book from the shelf -- that work I abhor -- then you also have exactly the same right and so does everyone else. And then we have no books left on the shelf for any of us." --Katherine Paterson, Author
Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.'''

#source_essay
prompt_3 = '''Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.'''

#source_essay
prompt_4 = ''' Read the last paragraph of the story. When they come back, Saeng vowed silently to herself, in the spring, when the snows melt and the geese return and this hibiscus is budding, then I will take that test again. Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.'''

#source_essay
prompt_5 = '''Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.'''

##source_essay
prompt_6 = '''Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.'''

prompt_7 = '''Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.
Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.'''

prompt_8 = '''We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part. '''


In [ ]:
df

,Unnamed: 0,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6
0,12255,20716,8,A long time ago when I was in third grade I h...,18.0,16.0,34.0,4.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,3.0
1,12256,20717,8,Softball has to be one of the single most gre...,21.0,26.0,46.0,5.0,4.0,5.0,4.0,4.0,4.0,6.0,5.0,6.0,6.0,5.0,5.0
2,12257,20718,8,"Some people like making people laugh, I love ...",15.0,20.0,40.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,5.0,4.0,4.0,4.0
3,12258,20719,8,"""LAUGHTER"" @CAPS1 I hang out with my friends...",12.0,20.0,30.0,3.0,3.0,3.0,3.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0
4,12259,20721,8,Well ima tell a story about the time i got @CA...,11.0,15.0,26.0,3.0,2.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,12973,21626,8,In most stories mothers and daughters are eit...,17.0,18.0,35.0,4.0,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,3.0
719,12974,21628,8,I never understood the meaning laughter is th...,15.0,17.0,32.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,3.0,4.0,4.0,4.0,3.0
720,12975,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20.0,26.0,40.0,4.0,4.0,4.0,4.0,4.0,4.0,6.0,5.0,5.0,5.0,5.0,5.0
721,12976,21630,8,Trippin' on fen...,20.0,20.0,40.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [ ]:
df_features = df.apply(lambda row: calculate_all_features(prompt_8, row['essay']), axis=1)
df_features

,0
0,"{'num_grammatical_errors': 61, 'adjacent_sente..."
1,"{'num_grammatical_errors': 17, 'adjacent_sente..."
2,"{'num_grammatical_errors': 21, 'adjacent_sente..."
3,"{'num_grammatical_errors': 70, 'adjacent_sente..."
4,"{'num_grammatical_errors': 115, 'adjacent_sent..."
...,...
718,"{'num_grammatical_errors': 19, 'adjacent_sente..."
719,"{'num_grammatical_errors': 44, 'adjacent_sente..."
720,"{'num_grammatical_errors': 14, 'adjacent_sente..."
721,"{'num_grammatical_errors': 6, 'adjacent_senten..."


In [ ]:
features_df = pd.json_normalize(df_features)

# Merge with the original DataFrame
df_with_features = pd.concat([df, features_df], axis=1)

# View the DataFrame with new features
df_with_features = df_with_features.drop(df_with_features.columns[0], axis=1)
df_with_features

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,...,sentence_length_std,discourse_connective_count,vocabulary_richness,semantic_relevance,flesch_reading_ease,flesch_kincaid_grade,gunning_fog_index,smog_index,automated_readability_index,coleman_liau_index
0,20716,8,A long time ago when I was in third grade I h...,18.0,16.0,34.0,4.0,4.0,4.0,4.0,...,9.002410,13,0.520505,0.707057,88.87,4.9,6.90,7.0,6.1,5.10
1,20717,8,Softball has to be one of the single most gre...,21.0,26.0,46.0,5.0,4.0,5.0,4.0,...,11.949837,16,0.635593,0.777121,78.93,8.7,11.37,8.9,10.8,6.04
2,20718,8,"Some people like making people laugh, I love ...",15.0,20.0,40.0,3.0,3.0,3.0,3.0,...,11.317015,16,0.578164,0.795551,74.83,8.2,9.62,8.5,9.1,6.15
3,20719,8,"""LAUGHTER"" @CAPS1 I hang out with my friends...",12.0,20.0,30.0,3.0,3.0,3.0,3.0,...,17.686580,14,0.595070,0.737897,81.46,7.7,9.68,7.0,8.3,4.65
4,20721,8,Well ima tell a story about the time i got @CA...,11.0,15.0,26.0,3.0,2.0,3.0,3.0,...,9.722998,17,0.522388,0.625690,88.60,7.1,9.98,4.2,7.7,3.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,21626,8,In most stories mothers and daughters are eit...,17.0,18.0,35.0,4.0,3.0,4.0,4.0,...,17.869146,19,0.627072,0.794858,65.09,12.0,13.56,10.3,14.4,7.03
719,21628,8,I never understood the meaning laughter is th...,15.0,17.0,32.0,3.0,3.0,4.0,3.0,...,9.989607,7,0.550000,0.791210,91.11,4.0,5.97,7.1,5.1,4.98
720,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20.0,26.0,40.0,4.0,4.0,4.0,4.0,...,17.083961,17,0.745509,0.766729,69.11,8.3,9.46,9.7,9.9,8.18
721,21630,8,Trippin' on fen...,20.0,20.0,40.0,4.0,4.0,4.0,4.0,...,8.045972,21,0.663793,0.819285,81.83,5.5,7.41,8.5,6.8,6.55


In [ ]:
df_with_features.to_csv("df_with_features_set_8.csv")

In [ ]:
jfeatures = df_with_features[["essay", "num_grammatical_errors", "adjacent_sentence_similarity","lexical_repetition_ratio",
             "unique_lemmas_count", "pronoun_count", "avg_sentence_length", "discourse_connective_count",
             "vocabulary_richness", "semantic_relevance", "flesch_reading_ease", "flesch_kincaid_grade",
             "gunning_fog_index", "smog_index", "automated_readability_index", "coleman_liau_index"
             ]]

scores = df_with_features[[ "rater1_trait1", "rater1_trait2", "rater1_trait3", "rater1_trait4",
              "rater2_trait1", "rater2_trait2", "rater2_trait3", "rater2_trait4",
              "domain1_score"]]
features, scores

(                                                  essay  \
 0     Patience is when your waiting .I was patience ...   
 1     I am not a patience person, like I can’t sit i...   
 2     One day I was at basketball practice and I was...   
 3     I going to write about a time when I went to t...   
 4     It can be very hard for somebody to be patient...   
 ...                                                 ...   
 1564  One time I was getting a cool @CAPS1 game it w...   
 1565  A patent person in my life is my mom. Aicason ...   
 1566  A time when someone else I know was patient wa...   
 1567  I hate weddings. I love when people get marrie...   
 1568  A few weeks ago, we had a garage sale and a mo...   
 
       num_grammatical_errors  adjacent_sentence_similarity  \
 0                         20                      0.723575   
 1                         10                      0.614597   
 2                         23                      0.769198   
 3                        

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [ ]:
train_features, test_features, train_scores, test_scores = train_test_split(features, scores, test_size=0.2, random_state=42)
len(train_features), len(test_features), len(train_scores), len(test_scores)

(1255, 314, 1255, 314)

In [ ]:
train_essays = train_features["essay"]
test_essays = test_features["essay"]
train_features_extract = train_features.drop("essay", axis=1)
test_features_extract = test_features.drop("essay", axis=1)

In [ ]:
scaled_features_train = scaler.fit_transform(train_features_extract)
scaled_features_test = scaler.transform(test_features_extract)

In [ ]:
scaled_features_train_df = pd.DataFrame(scaled_features_train, columns=train_features_extract.columns)
scaled_features_test_df = pd.DataFrame(scaled_features_test, columns=test_features_extract.columns)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
batch_size = 32
class EssayDataset(Dataset):
    def __init__(self, essays, scores, features, tokenizer, max_length=512):
        self.essays = essays.to_list()
        self.r1_scores = scores[['rater1_trait1', 'rater1_trait2', 'rater1_trait3', 'rater1_trait4']].values.tolist()
        self.r2_scores = scores[['rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4']].values.tolist()
        self.overall_scores = scores[['domain1_score']].values.tolist()
        self.features = features.values.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.essays)

    def __getitem__(self, idx):
        essay = self.essays[idx]
        r1_score = self.r1_scores[idx]
        r2_score = self.r2_scores[idx]
        overall_score = self.overall_scores[idx]
        feature = self.features[idx]

        tokens = self.tokenizer(essay, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return {'input_ids': tokens['input_ids'].squeeze().to(device),
                'attention_mask': tokens['attention_mask'].squeeze().to(device),
                'r1_scores': torch.tensor(r1_score, dtype=torch.float).to(device),
                'r2_scores': torch.tensor(r2_score, dtype=torch.float).to(device),
                'overall_score': torch.tensor(overall_score, dtype=torch.float).to(device),
                'features': torch.tensor(feature, dtype=torch.float).to(device)}

# Example usage:
train_dataset = EssayDataset(train_essays,
                             train_scores,
                             scaled_features_train_df,
                             tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


NameError: name 'train_essays' is not defined

In [ ]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class EssayDataset(Dataset):
#     def __init__(self, essays, scores, features, tokenizer, max_length=512):
#         self.essays = essays.to_list()
#         self.scores = scores.to_list()
#         self.features = features.to_dict('records')  # Convert DataFrame to list of dictionaries
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.essays)

#     def __getitem__(self, idx):
#         essay = self.essays[idx]
#         score = self.scores[idx]
#         feature_dict = self.features[idx]

#         tokens = self.tokenizer(essay, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

#         # Extract features and convert to tensor
#         feature_tensors = {k: torch.tensor(v, dtype=torch.float).to(device) for k, v in feature_dict.items()}

#         return {
#             'input_ids': tokens['input_ids'].squeeze().to(device),
#             'attention_mask': tokens['attention_mask'].squeeze().to(device),
#             'score': torch.tensor(score, dtype=torch.float).to(device),
#             **feature_tensors  # Include the features in the output
#         }

# # Example usage:
# # Assuming `features_df` is your DataFrame containing the features
# train_dataset = EssayDataset(train_essays, train_scores, features_df, tokenizer)
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
import torch.nn as nn
class BertForRegression(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        input_dim = config.hidden_size + 15
        self.r1_trait1_head = nn.Linear(input_dim, 1)
        self.r1_trait2_head = nn.Linear(input_dim, 1)
        self.r1_trait3_head = nn.Linear(input_dim, 1)
        self.r1_trait4_head = nn.Linear(input_dim, 1)

        self.r2_trait1_head = nn.Linear(input_dim, 1)
        self.r2_trait2_head = nn.Linear(input_dim, 1)
        self.r2_trait3_head = nn.Linear(input_dim, 1)
        self.r2_trait4_head = nn.Linear(input_dim, 1)
        self.overall_head = nn.Linear(input_dim, 1)

    def forward(self, input_ids, attention_mask=None, features = None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]

        # Concatenate features to the CLS output
        if features is not None:
            # Ensure features are of the right type and on the correct device
            features = features.to(cls_output.device, dtype=cls_output.dtype)
            combined_output = torch.cat((cls_output, features), dim=1)  # Concatenate along the feature dimension
        else:
            combined_output = cls_output  # If no features are provided, use only the CLS output
        r1_trait1 = self.r1_trait1_head(combined_output)
        r1_trait2 = self.r1_trait2_head(combined_output)
        r1_trait3 = self.r1_trait3_head(combined_output)
        r1_trait4 = self.r1_trait4_head(combined_output)

        r2_trait1 = self.r2_trait1_head(combined_output)
        r2_trait2 = self.r2_trait2_head(combined_output)
        r2_trait3 = self.r2_trait3_head(combined_output)
        r2_trait4 = self.r2_trait4_head(combined_output)
        overall = self.overall_head(combined_output)
        return r1_trait1, r1_trait2, r1_trait3, r1_trait4, r2_trait1, r2_trait2, r2_trait3, r2_trait4, overall

In [ ]:
from transformers import BertConfig
import torch.optim as optim

config = BertConfig.from_pretrained('bert-base-uncased')
model = BertForRegression.from_pretrained('bert-base-uncased', config=config).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        r1_scores = batch['r1_scores']
        r2_scores = batch['r2_scores']
        overall_scores = batch["overall_score"]
        features = batch["features"]


        targets = torch.cat((r1_scores, r2_scores, overall_scores), dim=1)


        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, features=features)

        model_outputs = torch.cat(outputs, dim=1)

        loss = criterion(model_outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

In [ ]:
test_dataset = EssayDataset(test_essays, test_scores, scaled_features_test_df, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [ ]:
import torch
from sklearn.metrics import cohen_kappa_score
import numpy as np

# Set model to evaluation mode
model.eval()

# Initialize variables for evaluation
all_predictions = []
all_targets = []
eval_loss = 0

# Disable gradient calculations for testing
with torch.no_grad():
    for batch in test_dataloader:
        # Extract inputs and move to the appropriate device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        r1_scores = batch['r1_scores'].to(device)
        r2_scores = batch['r2_scores'].to(device)
        overall_scores = batch["overall_score"].to(device)
        features = batch["features"].to(device)

        # Combine targets into a single tensor
        targets = torch.cat((r1_scores, r2_scores, overall_scores), dim=1)

        # Get predictions from the model
        outputs = model(input_ids, attention_mask=attention_mask, features=features)

        # Concatenate the outputs for the three traits and the overall score
        model_outputs = torch.cat(outputs, dim=1)

        # Calculate the evaluation loss for this batch
        eval_loss += criterion(model_outputs, targets).item()

        # Collect predictions and actual targets for further evaluation
        all_predictions.extend(model_outputs[:, -1].cpu().numpy())  # Assuming the last column is the overall score
        all_targets.extend(overall_scores.cpu().numpy())

# Calculate the average evaluation loss over all batches
avg_eval_loss = eval_loss / len(test_dataloader)
print(f'Validation Loss: {avg_eval_loss}')

# Convert predictions and targets to numpy arrays for QWK calculation
all_predictions = np.array(all_predictions).flatten()
all_targets = np.array(all_targets).flatten()

# Round predictions to the nearest integer for QWK
all_predictions_rounded = np.round(all_predictions)

# Calculate QWK for the overall score
qwk_overall = cohen_kappa_score(all_targets, all_predictions_rounded, weights='quadratic')
print(f'QWK Overall: {qwk_overall}')


Validation Loss: 0.9752898716096636
QWK Overall: 0.8213449518268734


In [ ]:
torch.save(model.state_dict(), 'model_with_handcrafted_feature.pt')
# When loading
# model.load_state_dict(torch.load('model_11.pt', map_location=device))


In [ ]:
model.eval()
index = 0
essay = test_essays.iloc[index]
score = test_scores.iloc[index]
feature = scaled_features_test_df.iloc[index]
def predict_essay(essay, model, tokenizer, device, features):
    model.eval()
    inputs = tokenizer(essay, return_tensors='pt', padding=True, truncation=True, max_length=512)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    feature_tensor = torch.tensor(features.values, dtype=torch.float32)
    features = feature_tensor.unsqueeze(0).to(device, dtype=torch.float)
    print(features.shape)

    with torch.no_grad():
        r1_trait1, r1_trait2, r1_trait3, r1_trait4, r2_trait1, r2_trait2, r2_trait3, r2_trait4, overall = model(input_ids=input_ids, attention_mask=attention_mask, features=features)
        predicted_r1_trait1 = r1_trait1.squeeze().cpu().numpy()  # Assuming single output per trait
        predicted_r1_trait2 = r1_trait2.squeeze().cpu().numpy()
        predicted_r1_trait3 = r1_trait3.squeeze().cpu().numpy()
        predicted_r1_trait4 = r1_trait4.squeeze().cpu().numpy()

        predicted_r2_trait1 = r2_trait1.squeeze().cpu().numpy()  # Assuming single output per trait
        predicted_r2_trait2 = r2_trait2.squeeze().cpu().numpy()
        predicted_r2_trait3 = r2_trait3.squeeze().cpu().numpy()
        predicted_r2_trait4 = r2_trait4.squeeze().cpu().numpy()

        predicted_overall = overall.squeeze().cpu().numpy()



    return predicted_r1_trait1, predicted_r1_trait2, predicted_r1_trait3, predicted_r1_trait4, predicted_r2_trait1,predicted_r2_trait2, predicted_r2_trait3,  predicted_r2_trait4, predicted_overall
predicted_r1_trait1, predicted_r1_trait2, predicted_r1_trait3, predicted_r1_trait4, predicted_r2_trait1,predicted_r2_trait2, predicted_r2_trait3,  predicted_r2_trait4, predicted_overall  = predict_essay(essay, model, tokenizer, device, feature)
print(test_essays.iloc[index])
print("predicted_r1: ideas, organization, style, convention \n ", predicted_r1_trait1, predicted_r1_trait2, predicted_r1_trait3, predicted_r1_trait4)
print("predicted_r2: ideas, organization, style, convention  \n ", predicted_r2_trait1,predicted_r2_trait2, predicted_r2_trait3,  predicted_r2_trait4)
print("predicted_r3: overall score \n ", predicted_overall)

print("\n")
print("actual: ", score)

torch.Size([1, 15])
My at saying your patient is. Your patient when you wait in line for something is being patient. Waiting your turn is being patience one time I was patient by waiting on the bench to play but I didn`t get to play until the fourth queiter with @NUM1 to go but you know what I did had patience. A lot of people have patient but some people just don`t have manner when they just don`t care and start the yell and screaming. Like popstars a lot of singers popstar and other famous people think they can cut in front of any one cause their rich but it they wern`t rich they would cut pop stars and movie stars just need to have patient. Am not saying all popstars and movie stars don`t have patient when they were little they never act like that.
predicted_r1: ideas, organization, style, convention 
  0.53180027 1.4078546 2.1636386 1.9615777
predicted_r2: ideas, organization, style, convention  
  0.9017922 1.6942227 1.8754803 2.0368257
predicted_r3: overall score 
  12.904104


a

In [ ]:
for i in range(10):
  essay = test_essays.iloc[i]
  score = test_scores.iloc[i]
  predicted = predict_essay(essay, model, tokenizer, device)
  print(test_essays.iloc[i])
  print("predicted: ", predicted, "actual: ", score, "\n")

TypeError: predict_essay() missing 1 required positional argument: 'features'